# PRIMARY SETUPS

In [1]:
# coding: utf-8

'''
Created on 21 мая 2016 г.

@author: keen
'''

from sklearn import datasets
import pickle

from scipy.sparse import csr_matrix
from sklearn import preprocessing
from numpy import ndarray
from numpy import asarray
from numpy.random import randint as rint
import datetime
from random import randint
import numpy
from sklearn.datasets import fetch_openml
#from sklearn.svm import SVC
import CO2_tree as co2t
import CO2_forest as co2f
from sklearn.metrics import accuracy_score

In [2]:
TRAIN_SIZE = 1000
TEST_SIZE = 1000
SHAKE = True
NJOBS = 5

# Load all data

In [3]:
data = {}

## MNIST

In [4]:
x, Y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
x = preprocessing.normalize(x, copy=False, axis = 0)
Y = Y.astype(int) + 1

x_train = x[:60000,:]
x_validate = x[60000:,:]

Y_train = Y[:60000]#[:6000]
Y_validate = Y[60000:]#[:3000]

data['MNIST'] = [[x_train,Y_train],[x_validate,Y_validate],(28,28,1)]

## USPS

In [5]:
x, Y = fetch_openml("USPS", version=1, return_X_y=True, as_frame=False)
x = preprocessing.normalize(x, copy=False, axis = 0)
Y = Y.astype(int) + 1

x_train = x[:7291,:]
x_validate = x[7291:,:]

Y_train = Y[:7291]#[:6000]
Y_validate = Y[7291:]#[:3000]
data['USPS'] = [[x_train,Y_train],[x_validate,Y_validate],(16,16,1)]

/home/keen/.pyenv/versions/3.6.2/lib/python3.6/site-packages/sklearn/datasets/_openml.py:854: UserWarning: Version 1 of dataset USPS is inactive, meaning that issues have been found in the dataset. Try using a newer version from this URL: https://api.openml.org/data/v1/download/18805612/USPS.arff
  data_description['url']))


## CIFAR-10

In [6]:
x, Y = fetch_openml("CIFAR_10", version=1, return_X_y=True, as_frame=False)
x = preprocessing.normalize(x, copy=False, axis = 0)
Y = Y.astype(int) + 1

x_train = x[:50000,:]
x_validate = x[50000:,:]

Y_train = Y[:50000]#[:6000]
Y_validate = Y[50000:]#[:3000]
data['CIFAR'] = [[x_train,Y_train],[x_validate,Y_validate],(32,32,3)]

## LETTER

In [7]:
x, Y = fetch_openml("letter", version=1, return_X_y=True, as_frame=False)
x = preprocessing.normalize(x, copy=False, axis = 0)
Y = preprocessing.LabelEncoder().fit_transform(Y).astype(int) + 1

x_train = x[:50000,:]
x_validate = x[50000:,:]

Y_train = Y[:50000]#[:6000]
Y_validate = Y[50000:]#[:3000]
data['letter'] = [[x_train,Y_train],[x_validate,Y_validate],(4,4,1)]

## SATIMAGE

In [8]:
x, Y = fetch_openml("satimage", version=1, return_X_y=True, as_frame=False)
x = preprocessing.normalize(x, copy=False, axis = 0)
Y =  preprocessing.LabelEncoder().fit_transform(Y).astype(int) + 1

x_train = x[:50000,:]
x_validate = x[50000:,:]

Y_train = Y[:50000]#[:6000]
Y_validate = Y[50000:]#[:3000]
data['satimage'] = [[x_train,Y_train],[x_validate,Y_validate],(8,8,1)]

# DEFINE ALL POSSIBLE TEST SCENARIOS

In [9]:
from deepforest import CascadeForestClassifier
from sklearn.ensemble import RandomForestClassifier

#TREE_D = [4,6,8,10]
#TREE_F = [0.1,0.2,0.3,0.4,0.5,0.6,0.7]
TREE_D = [4]
TREE_F = [0.2]


KERNEL = ['linear','gaussian']
KERNEL_PRUNE = [True,False]
#KERNEL_PRUNE_C = [0.1,1,10,100]
#KERNEL_PRUNE_N = [0.05,0.1,0.2,0.3]
KERNEL_PRUNE_C = [10]
KERNEL_PRUNE_N = [0.1]

n_estimators = 2

estimators=[]

for t in TREE_D:
    for f in TREE_F:
        tmp = []
        tmp.append('RF')
        tmp.append(t)
        tmp.append(f)
        tmp.append([RandomForestClassifier(max_depth=t,max_features=f,random_state=i,n_jobs=NJOBS) for i in range(n_estimators)])
        estimators.append(tmp)  
        
        for k in KERNEL:
            for pr in KERNEL_PRUNE:
                if pr:
                    for c in KERNEL_PRUNE_C:
                        for n in KERNEL_PRUNE_N:
                            tmp = []
                            tmp.append('KF')
                            tmp.append(t)
                            tmp.append(f)
                            tmp.append(k)
                            tmp.append("Pruned")
                            tmp.append(c)
                            tmp.append(n)
                            tmp.append([co2f.CO2_forest(C=5500, min_samples_leaf=4,dual=False,tol = 0.0001,max_iter=1000,kernel=k,max_deth=t,n_jobs=NJOBS,sample_ratio=1.0, feature_ratio = f,n_estimators=30,gamma=100,dropout_low=0.0,dropout_high=1.0,noise=0.,cov_dr=0.,criteria='gain',reinforced = True, prune_level = n,reC=c) for i in range(n_estimators)])                            
                            estimators.append(tmp) 
                    else:
                        tmp = []

                        tmp.append('KF')
                        tmp.append(t)
                        tmp.append(f)
                        tmp.append(k)
                        tmp.append("Not pruned") 
                        tmp.append([co2f.CO2_forest(C=5500, min_samples_leaf=4,dual=False,tol = 0.0001,max_iter=1000,kernel=k,max_deth=t,n_jobs=NJOBS,sample_ratio=1.0, feature_ratio = f,n_estimators=30,gamma=100,dropout_low=0.0,dropout_high=1.0,noise=0.,cov_dr=0.,criteria='gain') for i in range(n_estimators)])                            

                        estimators.append(tmp)  
                    
WIN = [True,False]

#DS_WIN_SZ = {
#    'MNIST':[10,12,14,16,18,20],
#    'USPS':[6,8,10],
#    'LETTER':[4],
#    'CIFAR':[10,12,14,16,18,20],
#    'SATIMAGE':[6,8]
#}

DS_WIN_SZ = {
    'MNIST':[10],
    'USPS':[6],
    'letter':[4],
    'CIFAR':[10],
    'SATIMAGE':[6]
}


# Do tests

In [10]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

def LOG(str_):
    with open("log.txt", "a") as f:
        if str_ != '\n':
            f.write(str(str_) + ";")
        else:    
            f.write(str_)
            
for d in data:
    x_train = data[d][0][0]
    Y_train = data[d][0][1]
    x_validate = data[d][1][0]
    Y_validate = data[d][1][1]
    if SHAKE:
        tr_idxs = numpy.random.permutation(x_train.shape[0])[:TRAIN_SIZE]         
        tst_idxs = numpy.random.permutation(x_validate.shape[0])[:TEST_SIZE]         
        x_train = x_train[tr_idxs]
        Y_train = Y_train[tr_idxs]
        x_validate = x_validate[tst_idxs]
        Y_validate = Y_validate[tst_idxs]
        
    if data[d][2][2] > 1:
        x_train = x_train.reshape((data[d][2][2],-1,data[d][2][0],data[d][2][1]))
        x_validate = x_validate.reshape((data[d][2][2],-1,data[d][2][0],data[d][2][1]))
    else:
        x_train = x_train.reshape((-1,data[d][2][0],data[d][2][1]))
        x_validate = x_validate.reshape((-1,data[d][2][0],data[d][2][1]))
        
    for w in WIN:
        for wss in DS_WIN_SZ[d]:
            for e_idx,e in enumerate(estimators):
                est = e[len(e) - 1]    
                model = CascadeForestClassifier(max_layers=3)
                model.set_estimator(est)                    
                if w and data[d][2][0] < wss:
                    model.set_win([wss,-1,-1],[2,1,1],[data[d][2][0],1,1],[data[d][2][1],1,1],[0,0,0],[2,0,0],[False,True,True])
                    model.fit(x_train, Y_train)#
                    pred = model.predict(x_validate)
                else: 
                    model.fit(x_train.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2]), Y_train)#
                    pred = model.predict(x_validate.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2]))

                for i in range(len(e) - 1):
                    LOG(e[i])                    
                print (d,w,wss,e[:len(e)-1])


                LOG(precision_score(Y_validate, pred,average='macro'))
                LOG(recall_score(Y_validate, pred,average='macro'))                        
                LOG(accuracy_score(Y_validate, pred))     

                LOG('\n')


[2022-08-02 13:37:14.002] Start to fit the model:
[2022-08-02 13:37:14.002] Fitting cascade layer = 0 
(1000, 20)
[2022-08-02 13:37:21.221] layer = 0  | Val Acc = 75.400 % | Elapsed = 7.219 s
[2022-08-02 13:37:21.235] Fitting cascade layer = 1 
(1000, 20)
[2022-08-02 13:37:27.726] layer = 1  | Val Acc = 72.500 % | Elapsed = 6.491 s
[2022-08-02 13:37:27.726] Early stopping counter: 1 out of 2
[2022-08-02 13:37:27.740] Fitting cascade layer = 2 
(1000, 20)
[2022-08-02 13:37:32.286] layer = 2  | Val Acc = 71.700 % | Elapsed = 4.546 s
[2022-08-02 13:37:32.286] Early stopping counter: 2 out of 2
[2022-08-02 13:37:32.286] Handling early stopping
[2022-08-02 13:37:32.287] The optimal number of layers: 1
[2022-08-02 13:37:32.289] Start to evalute the model:
[2022-08-02 13:37:32.300] Evaluating cascade layer = 0 
MNIST True 10 ['RF', 4, 0.2]
[2022-08-02 13:37:33.579] Start to fit the model:
[2022-08-02 13:37:33.579] Fitting cascade layer = 0 
(1000, 20)
[2022-08-02 13:40:48.778] layer = 0  | Va

/home/keen/.pyenv/versions/3.6.2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1000, 20)
[2022-08-02 13:58:56.296] layer = 0  | Val Acc = 11.100 % | Elapsed = 280.093 s
[2022-08-02 13:58:56.307] Fitting cascade layer = 1 
(1000, 20)
[2022-08-02 14:02:41.028] layer = 1  | Val Acc = 9.200 % | Elapsed = 224.721 s
[2022-08-02 14:02:41.028] Early stopping counter: 1 out of 2
[2022-08-02 14:02:41.033] Fitting cascade layer = 2 
(1000, 20)
[2022-08-02 14:06:29.335] layer = 2  | Val Acc = 11.200 % | Elapsed = 228.302 s
[2022-08-02 14:06:29.336] Reaching the maximum number of layers: 3
[2022-08-02 14:06:29.339] Start to evalute the model:
[2022-08-02 14:06:29.391] Evaluating cascade layer = 0 
[2022-08-02 14:06:57.180] Evaluating cascade layer = 1 
[2022-08-02 14:07:26.767] Evaluating cascade layer = 2 
MNIST True 10 ['KF', 4, 0.2, 'gaussian', 'Pruned', 10, 0.1]
[2022-08-02 14:07:57.200] Start to fit the model:
[2022-08-02 14:07:57.200] Fitting cascade layer = 0 


/home/keen/.pyenv/versions/3.6.2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1000, 20)
[2022-08-02 14:10:57.845] layer = 0  | Val Acc = 10.400 % | Elapsed = 180.645 s
[2022-08-02 14:10:57.847] Fitting cascade layer = 1 
(1000, 20)
[2022-08-02 14:14:05.149] layer = 1  | Val Acc = 10.300 % | Elapsed = 187.302 s
[2022-08-02 14:14:05.149] Early stopping counter: 1 out of 2
[2022-08-02 14:14:05.151] Fitting cascade layer = 2 
(1000, 20)
[2022-08-02 14:17:13.006] layer = 2  | Val Acc = 9.100 % | Elapsed = 187.855 s
[2022-08-02 14:17:13.006] Early stopping counter: 2 out of 2
[2022-08-02 14:17:13.006] Handling early stopping
[2022-08-02 14:17:13.007] The optimal number of layers: 1
[2022-08-02 14:17:13.009] Start to evalute the model:
[2022-08-02 14:17:13.019] Evaluating cascade layer = 0 
MNIST True 10 ['KF', 4, 0.2, 'gaussian', 'Not pruned']
[2022-08-02 14:17:38.432] Start to fit the model:
[2022-08-02 14:17:38.432] Fitting cascade layer = 0 


...

[2022-08-02 18:11:48.413] Start to fit the model:
[2022-08-02 18:11:48.413] Fitting cascade layer = 0 
(1000, 20)
[2022-08-02 18:25:55.650] layer = 0  | Val Acc = 9.700 % | Elapsed = 847.238 s
[2022-08-02 18:25:55.656] Fitting cascade layer = 1 
(1000, 20)
[2022-08-02 18:46:14.951] layer = 1  | Val Acc = 11.300 % | Elapsed = 1219.294 s
[2022-08-02 18:46:14.956] Fitting cascade layer = 2 
(1000, 20)
[2022-08-02 19:04:32.721] layer = 2  | Val Acc = 9.800 % | Elapsed = 1097.765 s
[2022-08-02 19:04:32.721] Early stopping counter: 1 out of 2
[2022-08-02 19:04:32.722] Reaching the maximum number of layers: 3
[2022-08-02 19:04:32.731] Start to evalute the model:
[2022-08-02 19:04:32.987] Evaluating cascade layer = 0 
[2022-08-02 19:06:40.213] Evaluating cascade layer = 1 
[2022-08-02 19:09:00.006] Evaluating cascade layer = 2 
CIFAR False 10 ['KF', 4, 0.2, 'gaussian', 'Pruned', 10, 0.1]


/home/keen/.pyenv/versions/3.6.2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[2022-08-02 19:11:51.948] Start to fit the model:
[2022-08-02 19:11:51.948] Fitting cascade layer = 0 
(1000, 20)
[2022-08-02 19:28:43.026] layer = 0  | Val Acc = 9.300 % | Elapsed = 1011.078 s
[2022-08-02 19:28:43.052] Fitting cascade layer = 1 
(1000, 20)
[2022-08-02 19:39:17.199] layer = 1  | Val Acc = 8.300 % | Elapsed = 634.146 s
[2022-08-02 19:39:17.199] Early stopping counter: 1 out of 2
[2022-08-02 19:39:17.204] Fitting cascade layer = 2 
(1000, 20)
[2022-08-02 19:49:59.434] layer = 2  | Val Acc = 11.100 % | Elapsed = 642.230 s
[2022-08-02 19:49:59.435] Reaching the maximum number of layers: 3
[2022-08-02 19:49:59.476] Start to evalute the model:
[2022-08-02 19:49:59.653] Evaluating cascade layer = 0 
[2022-08-02 19:52:01.583] Evaluating cascade layer = 1 
[2022-08-02 19:54:05.254] Evaluating cascade layer = 2 
CIFAR False 10 ['KF', 4, 0.2, 'gaussian', 'Not pruned']


/home/keen/.pyenv/versions/3.6.2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KeyError: 'letter'